In [1]:
from collections import defaultdict
import ftplib
import getpass

In [2]:
#import urllib3
#urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
# explain above, and warn

In [3]:
import pandas as pd

from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy.histories import HistoryClient
from bioblend.galaxy.datasets import DatasetClient

In [4]:
#Need to create a file with this
with open('../galaxy.api') as f:
    api_key = f.readline().rstrip()

In [5]:
server = 'https://bioinformatics.dbs.umt.edu'
ftp_server = 'bioinformatics.dbs.umt.edu'
ftp_user = 'tiago.antao@mso.umt.edu'
ftp_port = 8021
history_name = 'data_science_example'

In [6]:
ftp_pass = getpass.getpass()

········


In [7]:
gi = GalaxyInstance(url=server, key=api_key)
gi.verify = False
histories = gi.histories

In [8]:
for history in histories.get_histories():
    if history['name'] == history_name:
        histories.delete_history(history['id'])
    print(history['name'])

data_science_example
Jenya
Unnamed history
Marc
brian
Data Manager History (automatically created)
reference genomes
Unnamed history


In [9]:
ds_history = histories.create_history(history_name)

In [10]:
ftp = ftplib.FTP() 
ftp.connect(host=ftp_server,port=ftp_port)
ftp.login(user=ftp_user, passwd=ftp_pass)
f = open('../raw/part-3L.vcf.gz','rb')
ftp.set_pasv(False)  # explain
ftp.storbinary('STOR part-3L.vcf.gz', f)
f.close() 
ftp.close()

In [11]:
gi.tools.upload_from_ftp('part-3L.vcf.gz', ds_history['id'])

{'implicit_collections': [],
 'jobs': [{'create_time': '2016-08-10T17:12:39.076756',
   'exit_code': None,
   'id': '0e74a7c1bf7ec99d',
   'model_class': 'Job',
   'state': 'new',
   'tool_id': 'upload1',
   'update_time': '2016-08-10T17:12:39.120913'}],
 'output_collections': [],
 'outputs': [{'create_time': '2016-08-10T17:12:38.982612',
   'data_type': 'galaxy.datatypes.data.Text',
   'deleted': False,
   'file_ext': 'auto',
   'file_size': 0,
   'genome_build': '?',
   'hda_ldda': 'hda',
   'hid': 1,
   'history_content_type': 'dataset',
   'history_id': '7d83dea28f2ec7cc',
   'id': '56573e25b995be9d',
   'metadata_data_lines': None,
   'metadata_dbkey': '?',
   'misc_blurb': None,
   'misc_info': None,
   'model_class': 'HistoryDatasetAssociation',
   'name': 'part-3L.vcf.gz',
   'output_name': 'output0',
   'peek': '<table cellspacing="0" cellpadding="3"></table>',
   'purged': False,
   'state': 'queued',
   'tags': [],
   'update_time': '2016-08-10T17:12:39.020363',
   'uuid': '

In [12]:
contents = gi.histories.show_history(ds_history['id'], contents=True)
contents

[{'dataset_id': 'a6f7719d361bdab3',
  'deleted': False,
  'extension': 'auto',
  'hid': 1,
  'history_content_type': 'dataset',
  'history_id': '7d83dea28f2ec7cc',
  'id': '56573e25b995be9d',
  'name': 'part-3L.vcf.gz',
  'purged': False,
  'state': 'running',
  'type': 'file',
  'type_id': 'dataset-56573e25b995be9d',
  'url': '/api/histories/7d83dea28f2ec7cc/contents/56573e25b995be9d',
  'visible': True}]

In [13]:
def summarize_contents(contents):
    summary = defaultdict(list)
    for item in contents:
        summary['íd'].append(item['id'])
        summary['híd'].append(item['hid'])
        summary['name'].append(item['name'])
        summary['type'].append(item['type'])
        summary['extension'].append(item['extension'])
    return pd.DataFrame.from_dict(summary)

In [14]:
summarize_contents(contents)

,extension,híd,name,type,íd
0,auto,1,part-3L.vcf.gz,file,56573e25b995be9d


In [15]:
vcf_ds = contents[0]
vcf_ds

{'dataset_id': 'a6f7719d361bdab3',
 'deleted': False,
 'extension': 'auto',
 'hid': 1,
 'history_content_type': 'dataset',
 'history_id': '7d83dea28f2ec7cc',
 'id': '56573e25b995be9d',
 'name': 'part-3L.vcf.gz',
 'purged': False,
 'state': 'running',
 'type': 'file',
 'type_id': 'dataset-56573e25b995be9d',
 'url': '/api/histories/7d83dea28f2ec7cc/contents/56573e25b995be9d',
 'visible': True}

In [16]:
gi.tools.get_tools()

[{'config_file': '/galaxy-central/tools/data_source/ucsc_tablebrowser.xml',
  'description': 'table browser',
  'id': 'ucsc_table_direct1',
  'labels': [],
  'link': '/tool_runner/data_source_redirect?tool_id=ucsc_table_direct1',
  'min_width': '800',
  'model_class': 'DataSourceTool',
  'name': 'UCSC Main',
  'panel_section_id': 'getext',
  'panel_section_name': 'Get Data',
  'target': '_top',
  'version': '1.0.0'},
 {'config_file': '/galaxy-central/tools/maf/maf_to_bed.xml',
  'description': 'Converts a MAF formatted file to the BED format',
  'id': 'MAF_To_BED1',
  'labels': [],
  'link': '/tool_runner?tool_id=MAF_To_BED1',
  'min_width': -1,
  'model_class': 'Tool',
  'name': 'MAF to BED',
  'panel_section_id': 'convert',
  'panel_section_name': 'Convert Formats',
  'target': 'galaxy_main',
  'version': '1.0.0'},
 {'config_file': '/shed_tools/toolshed.g2.bx.psu.edu/repos/devteam/table_annovar/08b003ee9db7/table_annovar/table_annovar.xml',
  'description': 'with functional informati

In [17]:
vcf2tsv = gi.tools.get_tools(name='VCFtoTab-delimited:')[0]
gi.tools.show_tool(vcf2tsv['id'], io_details=True, link_details=True)

{'config_file': '/shed_tools/toolshed.g2.bx.psu.edu/repos/devteam/vcf2tsv/d6c944f1581a/vcf2tsv/vcf2tsv.xml',
 'description': 'Convert VCF data into TAB-delimited format',
 'id': 'toolshed.g2.bx.psu.edu/repos/devteam/vcf2tsv/vcf2tsv/0.0.3',
 'inputs': [{'argument': None,
   'edam_formats': ['format_3016'],
   'extensions': ['vcf'],
   'help': '',
   'hidden': False,
   'is_dynamic': False,
   'label': 'Select VCF dataset to convert',
   'model_class': 'DataToolParameter',
   'multiple': False,
   'name': 'input',
   'optional': False,
   'options': {'hda': [], 'hdca': []},
   'type': 'data'},
  {'argument': None,
   'falsevalue': '',
   'help': '-g option',
   'hidden': False,
   'is_dynamic': False,
   'label': 'Report data per sample',
   'model_class': 'BooleanToolParameter',
   'name': 'g_option',
   'optional': False,
   'truevalue': '-g',
   'type': 'boolean',
   'value': True},
  {'argument': None,
   'display': None,
   'help': '-n option',
   'hidden': False,
   'is_dynamic': F

In [18]:
ds_history

{'annotation': None,
 'contents_url': '/api/histories/7d83dea28f2ec7cc/contents',
 'create_time': '2016-08-10T17:12:34.918207',
 'deleted': False,
 'empty': True,
 'genome_build': None,
 'id': '7d83dea28f2ec7cc',
 'importable': False,
 'model_class': 'History',
 'name': 'data_science_example',
 'published': False,
 'purged': False,
 'size': 0,
 'slug': None,
 'state': 'new',
 'state_details': {'discarded': 0,
  'empty': 0,
  'error': 0,
  'failed_metadata': 0,
  'new': 0,
  'ok': 0,
  'paused': 0,
  'queued': 0,
  'running': 0,
  'setting_metadata': 0,
  'upload': 0},
 'state_ids': {'discarded': [],
  'empty': [],
  'error': [],
  'failed_metadata': [],
  'new': [],
  'ok': [],
  'paused': [],
  'queued': [],
  'running': [],
  'setting_metadata': [],
  'upload': []},
 'tags': [],
 'update_time': '2016-08-10T17:12:34.918225',
 'url': '/api/histories/7d83dea28f2ec7cc',
 'user_id': 'f2db41e1fa331b3e',
 'username_and_slug': None}

In [19]:
def dataset_to_param(dataset):
    return dict(src='hda', id=dataset['id'])

In [20]:
tool_inputs = {'input': dataset_to_param(vcf_ds), 'g_option': True, 'null_filter': ''}
#hid!
print(tool_inputs)
gi.tools.run_tool(ds_history['id'], vcf2tsv['id'], tool_inputs=tool_inputs)


{'null_filter': '', 'input': {'id': '56573e25b995be9d', 'src': 'hda'}, 'g_option': True}


{'implicit_collections': [],
 'jobs': [{'create_time': '2016-08-10T17:13:05.271059',
   'exit_code': None,
   'id': 'd49aa4a2fce5bb0a',
   'model_class': 'Job',
   'state': 'new',
   'tool_id': 'toolshed.g2.bx.psu.edu/repos/devteam/vcf2tsv/vcf2tsv/0.0.3',
   'update_time': '2016-08-10T17:13:05.271072'}],
 'output_collections': [],
 'outputs': [{'create_time': '2016-08-10T17:13:05.217759',
   'data_type': 'galaxy.datatypes.tabular.Tabular',
   'deleted': False,
   'file_ext': 'tabular',
   'file_size': 0,
   'genome_build': '?',
   'hda_ldda': 'hda',
   'hid': 2,
   'history_content_type': 'dataset',
   'history_id': '7d83dea28f2ec7cc',
   'id': '2700ea79f26a88c2',
   'metadata_column_names': None,
   'metadata_column_types': None,
   'metadata_columns': None,
   'metadata_comment_lines': None,
   'metadata_data_lines': None,
   'metadata_dbkey': '?',
   'metadata_delimiter': '\t',
   'misc_blurb': 'queued',
   'misc_info': None,
   'model_class': 'HistoryDatasetAssociation',
   'name':